In [ ]:
import dask
import numpy as np

In [ ]:
# from dask.distributed import Client

# client = Client(n_workers=8)
# client

In [ ]:
import dask.dataframe as dd

rankings = dd.read_csv(
    "data/AMPLab/1node/rankings/*.csv",
    header=None,
    names=["pageURL", "pageRank", "avgDuration"],
)
rankings.info()

In [ ]:
%%time
query_1 = rankings[rankings.pageRank > 1000]
len(query_1)

In [ ]:
uservisits = dd.read_csv(
    "data/AMPLab/1node/uservisits/*.csv",
    header=None,
    names=["sourceIP", "destinationURL", "visitDate", "adRevenue", "UserAgent", "cCode", "lCode", "searchWord", "duration"],
    parse_dates=["visitDate"],
)
uservisits.info()

In [ ]:
%%time
query_2 = (
    uservisits
        .groupby(uservisits.sourceIP.str.slice(0, 7))
        .adRevenue
        .sum()
)
query_2.compute()

In [ ]:
%%time
query_3 = (rankings
    .merge(uservisits[
              (uservisits.visitDate > "1980-01-01")
            & (uservisits.visitDate < "1980-04-01")
           ],
           left_on="pageURL",
           right_on="destinationURL",
           how="left",
    )
    .groupby("sourceIP")
    .agg({"pageRank": "mean", "adRevenue": "sum"})
    .sort_values("adRevenue", ascending=False)
    .head(1)
)
query_3.compute()